In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO
from datetime import datetime
from urllib.request import Request, urlopen

In [2]:
class AASJobTable:
    def __init__(self,link:str):
        self.link = link
        self.headers = {'User-Agent': 'Mozilla/5.0'}

    def retrieve_table(self):
        req = Request(self.link, headers=self.headers)
        webpage = urlopen(req).read()
        
        soup = BeautifulSoup(webpage, "html.parser")
        table = soup.find_all("table")
        df = pd.read_html(StringIO(str(table)))[0]

        self.table = df.copy(deep=True)
            
    def filter_by_date(self, min_date:str, max_date:str='2100-01-01', date_kw:str='Deadline'):
        self.table[date_kw] = pd.to_datetime(self.table[date_kw],format='%Y/%m/%d')
        
        mask = (self.table[date_kw] > min_date) & (self.table[date_kw] <= max_date)
        self.table = self.table.loc[mask]
        self.table = self.table.sort_values(by=date_kw)

In [3]:
link = 'https://aas.org/jobregister?f%5B0%5D=category%3A511'

today = datetime.today().strftime('%Y-%m-%d')
maxdate = '2024-11-15'

In [4]:
T = AASJobTable(link)
T.retrieve_table()
T.filter_by_date(today,maxdate)

T.table

,Title,Institution,Location(s),Posted,Deadline
162,Project Research Fellow (Project Researcher on...,National Astronomical Observatory of Japan (NAOJ),"Mitaka, Tokyo Japan",2024/09/24,2024-11-05
188,Brinson Prize Fellowship 2025,Space Telescope Science Institute,"Baltimore, MD United States",2024/09/10,2024-11-07
144,Post-doctoral Positions and Fellowships,Perimeter Institute for Theoretical Physics,Waterloo ON Canada,2024/10/01,2024-11-08
173,THEA Postdoctoral Research Fellowship in Theor...,Columbia University,"New York, NY United States",2024/09/19,2024-11-08
61,"Flatiron Research Fellow, Learning the Univers...",Simons Foundation | Flatiron Institute,"New York, NY United States",2024/10/21,2024-11-08
106,"ASIAA Postdoctoral Positions in Observation, T...",Academia Sinica Institute of Astronomy and Ast...,"Da'an District, Taipei City Taiwan",2024/10/11,2024-11-14
112,Postdoctoral Fellowships in Astrophysics,University of Modena and Reggio Emilia,Modena MO Italy,2024/10/10,2024-11-14
191,Postdoctoral Fellowships in Astrophysics,Max Planck Institute for Astrophysics,Garching near Munich Germany,2024/09/06,2024-11-15
202,Simons Collaboration on Extreme Electrodynamic...,Stanford University,NaN,2024/09/03,2024-11-15
180,Postdoctoral Research Associate,University of Arizona,NaN,2024/09/16,2024-11-15
